In [1]:
import regex as re   
from tldextract import extract
import ssl
import socket
from bs4 import BeautifulSoup
import urllib.request
import whois
import datetime
from urllib.parse import urlparse,urlencode

In [2]:

def url_having_ip(url):
    match = re.search(
        '(([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.'
        '([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\/)|'  # IPv4
        '((0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\/)'  # IPv4 in hexadecimal
        '(?:[a-fA-F0-9]{1,4}:){7}[a-fA-F0-9]{1,4}', url)  # Ipv6
    if match:
        # print match.group()
        return -1
    else:
        # print 'No matching pattern found'
        return 1


   

In [3]:
def url_length(url):
    length=len(url)
    if(length<54):
        return 1
    elif(54<=length<=75):
        return 0
    else:
        return -1



In [4]:
def url_short(url):
    #ongoing
     match = re.search('bit\.ly|goo\.gl|shorte\.st|go2l\.ink|x\.co|ow\.ly|t\.co|tinyurl|tr\.im|is\.gd|cli\.gs|'
                      'yfrog\.com|migre\.me|ff\.im|tiny\.cc|url4\.eu|twit\.ac|su\.pr|twurl\.nl|snipurl\.com|'
                      'short\.to|BudURL\.com|ping\.fm|post\.ly|Just\.as|bkite\.com|snipr\.com|fic\.kr|loopt\.us|'
                      'doiop\.com|short\.ie|kl\.am|wp\.me|rubyurl\.com|om\.ly|to\.ly|bit\.do|t\.co|lnkd\.in|'
                      'db\.tt|qr\.ae|adf\.ly|goo\.gl|bitly\.com|cur\.lv|tinyurl\.com|ow\.ly|bit\.ly|ity\.im|'
                      'q\.gs|is\.gd|po\.st|bc\.vc|twitthis\.com|u\.to|j\.mp|buzurl\.com|cutt\.us|u\.bb|yourls\.org|'
                      'x\.co|prettylinkpro\.com|scrnch\.me|filoops\.info|vzturl\.com|qr\.net|1url\.com|tweez\.me|v\.gd|'
                      'tr\.im|link\.zip\.net',
                      url)
     if match:
        return -1
     else:
        return 1


    


In [5]:
def having_at_symbol(url):
    match = re.search('@', url)
    if match:
        return -1
    else:
        return 1
 

In [6]:
def doubleSlash(url):
    #ongoing
    # since the position starts from, we have given 6 and not 7 which is according to the document
    list = [x.start(0) for x in re.finditer('//', url)]
    if list[len(list) - 1] > 6:
        return -1
    else:
        return 1


     

In [7]:
def prefix_suffix(url):
    subDomain, domain, suffix = extract(url)
    
    if(domain.count('-')):
        return -1
    else:
        return 1
a=prefix_suffix("https://drive.google.com/drive/my-drive")
a

1

In [8]:
def sub_domain(url):
    # Here, instead of greater than 1 we will take greater than 3 since the greater than 1 conition is when www and
    # country domain dots are skipped
    # Accordingly other dots will increase by 1
    if url_having_ip(url) == -1:
        match = re.search(
            '(([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.'
            '([01]?\\d\\d?|2[0-4]\\d|25[0-5]))|(?:[a-fA-F0-9]{1,4}:){7}[a-fA-F0-9]{1,4}',
            url)
        pos = match.end(0)
        url = url[pos:]
    list = [x.start(0) for x in re.finditer('\.', url)]
    if len(list) <= 3:
        return 1
    elif len(list) == 4:
        return 0
    else:
        return -1
    
    
a=sub_domain("https://drive.google.com/drive/my-drive")
a



1

In [9]:
def SSLfinal_State(url):
    try:
#check wheather contains https       
        if(re.search('^https',url)):
            usehttps = 1
        else:
            usehttps = 0
        #print(usehttps)
#getting the certificate issuer to later compare with trusted issuer 
        #getting host name
        subDomain, domain, suffix = extract(url)
        host_name = domain + "." + suffix
        context = ssl.create_default_context()
        sct = context.wrap_socket(socket.socket(), server_hostname = host_name)
        sct.connect((host_name, 443))
        certificate = sct.getpeercert()
        issuer = dict(x[0] for x in certificate['issuer'])
        certificate_Auth = str(issuer['commonName'])
        certificate_Auth = certificate_Auth.split()
        if(certificate_Auth[0] == "Network" or certificate_Auth == "Deutsche"):
            certificate_Auth = certificate_Auth[0] + " " + certificate_Auth[1]
        else:
            certificate_Auth = certificate_Auth[0] 
        trusted_Auth = ['Comodo','Symantec','GoDaddy','GlobalSign','DigiCert','StartCom','Entrust','Verizon','Trustwave','Unizeto','Buypass','QuoVadis','Deutsche Telekom','Network Solutions','SwissSign','IdenTrust','Secom','TWCA','GeoTrust','Thawte','Doster','VeriSign']        
#getting age of certificate
        startingDate = str(certificate['notBefore'])
        endingDate = str(certificate['notAfter'])
        startingYear = int(startingDate.split()[3])
        endingYear = int(endingDate.split()[3])
        Age_of_certificate = endingYear-startingYear
        
#checking final conditions
        if((usehttps==1) and (certificate_Auth in trusted_Auth) and (Age_of_certificate>=1) ):
            return 1 #legitimate
        elif((usehttps==1) and (certificate_Auth not in trusted_Auth)):
            return 0 #suspicious
        else:
            return -1 #phishing
        
    except Exception as e:
        
        return -1  
    
    
a=SSLfinal_State("http://133.130.103.10/1/")
a

-1

In [10]:
def domain_registration(url):
   
    try:
    
        hostname = url
        h = [(x.start(0), x.end(0)) for x in re.finditer('https://|http://|www.|https://www.|http://www.', hostname)]
        z = int(len(h))
        if z != 0:
                y = h[0][1]
                hostname = hostname[y:]
                h = [(x.start(0), x.end(0)) for x in re.finditer('/', hostname)]
                z = int(len(h))
                if z != 0:
                    hostname = hostname[:h[0][0]]
        domain = whois.whois(hostname)
    
        
        expiration_date = domain.expiration_date
        a=expiration_date.year
        b=expiration_date.day
        c=expiration_date.month
        exp=datetime.date(a,c,b)
        registration_length = 0
        today=datetime.datetime.now().date()
    # Some domains do not have expiration dates. The application should not raise an error if this is the case.
        if expiration_date:
            registration_length = abs((exp - today).days)
        if registration_length / 365 <= 1:
            return -1
        else:
            return 1
    except:
        return -1
a=domain_registration("http://133.130.103.10/1/")
a

-1

In [11]:
def favicon(url):
    try:
        subDomain, domain, suffix = extract(url)
        websiteDomain = domain
        
        opener = urllib.request.urlopen(url).read()
        soup = BeautifulSoup(opener, 'lxml')
        for head in soup.find_all('head'):
            
            for head.link in soup.find_all('link', href=True):
                dots = [x.start(0) for x in re.finditer('\.', head.link['href'])]
                if url in head.link['href'] or len(dots) == 1 or domain in head.link['href']:
                    q=0
                else:
                    q=1
                    return -1
                    break
            if q==0:
                return 1
    except:
        return -1

def port(url):
    #ongoing
    return 0

In [12]:
def https_token(url):
    match = re.search('https://|http://', url)
    if match.start(0) == 0:
        url = url[match.end(0):]
    match = re.search('http|https', url)
    if match:
        return -1
    else:
        return 1


In [13]:
def request_url(url):
    try:
        subDomain, domain, suffix = extract(url)
        websiteDomain = domain
        
        opener = urllib.request.urlopen(url).read()
        soup = BeautifulSoup(opener, 'lxml')
        imgs = soup.findAll('img', src=True)
        total = len(imgs)
        
        linked_to_same = 0
        avg =0
        for image in imgs:
            subDomain, domain, suffix = extract(image['src'])
            imageDomain = domain
            if(websiteDomain==imageDomain or imageDomain==''):
                linked_to_same = linked_to_same + 1
        vids = soup.findAll('video', src=True)
        total = total + len(vids)
        
        for video in vids:
            subDomain, domain, suffix = extract(video['src'])
            vidDomain = domain
            if(websiteDomain==vidDomain or vidDomain==''):
                linked_to_same = linked_to_same + 1
        linked_outside = total-linked_to_same
        if(total!=0):
            avg = linked_outside/total
            
        if(avg<0.22):
            return 1
        elif(0.22<=avg<=0.61):
            return 0
        else:
            return -1
    except:
        return -1
a=request_url("http://www.python.org")
a


1

In [14]:
def url_of_anchor(url):
    try:
        subDomain, domain, suffix = extract(url)
        websiteDomain = domain
        
        opener = urllib.request.urlopen(url).read()
        soup = BeautifulSoup(opener, 'lxml')
        z=soup.find_all('a')
        total = len(z)
        #print(total)
        linked_to_same = 0
        linked_outside=0
        avg = 0
        for link in soup.find_all('a'):
            a=link.get('href')
            subDomain, domain, suffix = extract(a)
            anchorDomain = domain
            if(websiteDomain==anchorDomain or anchorDomain==''):
                linked_to_same = linked_to_same + 1
        linked_outside = total-linked_to_same
        #print(linked_outside)
        if(total!=0):
            avg = linked_outside/total
           # print(avg)
            
        if(avg<0.31):
           #print(avg)
            return 1
        elif(0.31<=avg<=0.67):
            return 0
        else:
            return -1
    except:
        return -1
a=url_of_anchor("http://localhost:8888/notebooks/prjt/f_extraction.ipynb")
a

0

0.0

In [15]:
def Links_in_tags(url):
    try:
        subDomain, domain, suffix = extract(url)
        opener = urllib.request.urlopen(url).read()
        soup = BeautifulSoup(opener, 'lxml')
        
        i = 0
        success = 0
        for link in soup.find_all('link', href=True):
            #print(link.get('href'))
            dots = [x.start(0) for x in re.finditer('\.', link['href'])]
            #print(dots)
            if url in link['href'] or domain in link['href'] or len(dots) == 1:
                success = success + 1
            i = i + 1
        for script in soup.find_all('script', src=True):
            #print(script.get('src'))
            dots = [x.start(0) for x in re.finditer('\.', script['src'])]
            #print(dots)
            if url in script['src'] or domain in script['src'] or len(dots) == 1:
                success = success + 1
            i = i + 1
        for meta in soup.find_all('meta', content=True):
            #print(meta.get('content'))
            dots = [x.start(0) for x in re.finditer('\.', meta['content'])]
            #print(dots)
            if url in meta['content'] or domain in meta['content'] or len(dots) == 1:
                success = success + 1
            i = i + 1
        
        if(i!=0):
            avg = success/i
            #print(avg)

        if(avg<0.17):
            return 1
        elif(0.17<=avg<=0.81):
            return 0
        else:
            return -1        
    except:        
        return -1
a=Links_in_tags("https://www.pythonforbeginners.com/beautifulsoup/beautifulsoup-4-python")
a

0

In [16]:
def sfh(url):
    try:
        subDomain, domain, suffix = extract(url)
        opener = urllib.request.urlopen(url).read()
        soup = BeautifulSoup(opener, 'lxml')
        for form in soup.find_all('form', action=True):
        #print(form)
            if form['action'] == "" or form['action'] == "about:blank":
                return -1
            elif url not in form['action'] and domain not in form['action']:
                return 0
            else:
                return 1
    except:
        return -1
    
    
sfh("https://www.pythonforbeginners.com/beautifulsoup/beautifulsoup-4-python")

0

In [18]:
def email_submit(url):
    try:
        opener = urllib.request.urlopen(url).read()
        soup = BeautifulSoup(opener, 'lxml')
        for form in soup.find_all('form', action=True):
            if "mailto:" in form['action']:
                return -1
            else:
                return 1
    except:
        return -1


In [19]:
def abnormal_url(url):
    hostname = url
    h = [(x.start(0), x.end(0)) for x in re.finditer('https://|http://|www.|https://www.|http://www.', hostname)]
    z = int(len(h))
    if z != 0:
        y = h[0][1]
        hostname = hostname[y:]
        h = [(x.start(0), x.end(0)) for x in re.finditer('/', hostname)]
        z = int(len(h))
        if z != 0:
            hostname = hostname[:h[0][0]]
    match = re.search(hostname, url)
    if match:
        return 1
    else:
        return -1

def redirect(url):
    #ongoing
     #If the url has symbol(//) after protocol then such URL is to be classified as phishing """
        if "//" in urlparse(url).path:
            return 1            # phishing
        else:
            return 0            # legitimate
    #return 0
a=redirect("https://drive.google.com/drive/my-drive")
a

def on_mouseover(url):
    #ongoing
    return 0

def rightClick(url):
    #ongoing
    return 0

def popup(url):
    #ongoing
    return 0

def iframe(url):
    #ongoing
    return 0


In [20]:
 
def age_of_domain(url):
    try:
        hostname = url
        h = [(x.start(0), x.end(0)) for x in re.finditer('https://|http://|www.|https://www.|http://www.', hostname)]
        z = int(len(h))
        if z != 0:
                y = h[0][1]
                hostname = hostname[y:]
                h = [(x.start(0), x.end(0)) for x in re.finditer('/', hostname)]
                z = int(len(h))
                if z != 0:
                    hostname = hostname[:h[0][0]]
        domain = whois.whois(hostname)
     
    
        creation_date = domain.creation_date
        expiration_date = domain.expiration_date
        if ((expiration_date is None) or (creation_date is None)):
            return 1
        elif ((type(expiration_date) is list) or (type(creation_date) is list)):
            return 0
        else:
            ageofdomain = abs((expiration_date - creation_date).days)
        if ((ageofdomain/30) < 6):
            return -1
        else:
            return 1
    except:
        return -1

In [21]:
def dns(url):
    #ongoing
    return 0

def web_traffic(url):
    try:
        rank = BeautifulSoup(urllib.request.urlopen("http://data.alexa.com/data?cli=10&dat=s&url=" + url).read(), "xml").find("REACH")['RANK']
    except TypeError:
        return -1
    rank= int(rank)
    if (rank<100000):
        return 1
    else:
        return 0

def page_rank(url):
    #ongoing
    return 0

def google_index(url):
    site = re.search(url, 5)
    if site:
        return 1
    else:
        return -1


def links_pointing(url):
    #ongoing
    return 0

def statistical(url):
    #ongoing
    return 0

TypeError: expected string or buffer

In [25]:


def main(url):
    import pandas as pd


    
    
    check = [url_having_ip(url),url_length(url),url_short(url),having_at_symbol(url),doubleSlash(url),prefix_suffix(url),sub_domain(url),SSLfinal_State(url),domain_registration(url),favicon(url),https_token(url),request_url(url),url_of_anchor(url),Links_in_tags(url),sfh(url),email_submit(url),abnormal_url(url),redirect(url),age_of_domain(url),web_traffic(url)]
    check1=pd.DataFrame(check)
    
     
    return check1
url="http://pradochaui.com.br/criptografado1/index1.php"
q=main(url)
print(q)
type(q)


    0
0   1
1   1
2   1
3   1
4   1
5   1
6   1
7  -1
8  -1
9  -1
10  1
11 -1
12 -1
13 -1
14 -1
15 -1
16  1
17  0
18  1
19 -1


pandas.core.frame.DataFrame

0

In [30]:
print("ac")

ac


In [12]:
url="https://drive.google.com/drive/my-drive"
subDomain, domain, suffix = extract(url)

In [13]:
subDomain

'drive'

In [14]:
domain

'google'

In [15]:
suffix

'com'

In [18]:
expiration_date = domain.expiration_date
today = time.strftime('%Y-%m-%d')
today = datetime.strptime(today, '%Y-%m-%d')

AttributeError: 'str' object has no attribute 'expiration_date'

In [19]:
w = whois.whois(url)
updated = w.updated_date
exp = w.expiration_date
length = (exp[0]-updated[0]).days

timeout: timed out

In [ ]:
dns = 1
    try:
        domain = whois.query(hostname)
    except:
dns = -1

In [51]:
hostname = url
h = [(x.start(0), x.end(0)) for x in re.finditer('https://|http://|www.|https://www.|http://www.', hostname)]
z = int(len(h))
if z != 0:
        y = h[0][1]
        hostname = hostname[y:]
        h = [(x.start(0), x.end(0)) for x in re.finditer('/', hostname)]
        z = int(len(h))
        if z != 0:
                hostname = hostname[:h[0][0]]

In [52]:
h

[(16, 17), (22, 23)]

In [53]:
z

2

In [54]:
hostname

'drive.google.com'

In [55]:
dns = 1
try:
    domain = whois.query(hostname)
    
except:
    dns = -1

In [56]:
domain

'google'

In [57]:
expiration_date = domain.expiration_date

AttributeError: 'str' object has no attribute 'expiration_date'

In [50]:
w = whois.whois("http://localhost:8888/notebooks/prjt/f_extraction.ipynb")
w.expiration_date

In [116]:
url="http://webscraping.com/"

In [63]:
hostname = url
h = [(x.start(0), x.end(0)) for x in re.finditer('https://|http://|www.|https://www.|http://www.', hostname)]
z = int(len(h))
if z != 0:
        y = h[0][1]
        hostname = hostname[y:]
        h = [(x.start(0), x.end(0)) for x in re.finditer('/', hostname)]
        z = int(len(h))
        if z != 0:
                hostname = hostname[:h[0][0]]


In [71]:
hostname

'webscraping.com'

In [80]:
w = whois.whois(hostname)
c=w.expiration_date

In [83]:
import time
today = time.strftime('%Y-%m-%d')
#today = datetime.strptime(today, '%Y-%m-%d')

registration_length = 0
    # Some domains do not have expiration dates. The application should not raise an error if this is the case.
if w.expiration_date:
    registration_length = abs((w.expiration_date- today).days)
if registration_length / 365 <= 1:
    return -1
else:
    return 1

TypeError: unsupported operand type(s) for -: 'datetime.datetime' and 'str'

In [117]:
try:
    w = whois.whois(url)
    updated = w.updated_date
    exp = w.expiration_date
    d1 = datetime.strptime(exp, "%Y-%m-%d")
    d1 = datetime.strptime(updated, "%Y-%m-%d")
    length = (exp-updated).days
    if(length<=365):
        print("1") 
    else:
        print("-1")
except:
        print("0")

0


In [92]:
updated


[datetime.datetime(2013, 8, 20, 8, 8, 30),
 datetime.datetime(2013, 8, 20, 8, 8, 29)]

In [97]:
length = exp - updated


TypeError: unsupported operand type(s) for -: 'datetime.datetime' and 'list'

In [100]:
d1 = datetime.datetime.strptime(exp, "%Y-%m-%d")
d1 = datetime.strptime(updated, "%Y-%m-%d")

TypeError: strptime() argument 1 must be str, not datetime.datetime

In [113]:
d1=datetime.date.today()

In [114]:
d2=datetime.date.today()

In [115]:
abs((d2 - d1).days)

0

In [107]:
d1 = datetime.strptime(d1, "%Y-%m-%d")
    d2 = datetime.strptime(d2, "%Y-%m-%d")
    return abs((d2 - d1).days)


IndentationError: unexpected indent (<ipython-input-107-dc195e41356b>, line 2)

In [109]:
from datetime import date 
def numOfDays(date1, date2): 
    return (date2-date1).days 
      
# Driver program 
date1 = date(2018, 12, 13) 
date2 = date(2019, 2, 25) 
print(numOfDays(date1, date2), "days") 

74 days


In [118]:
updated


[datetime.datetime(2013, 8, 20, 8, 8, 30),
 datetime.datetime(2013, 8, 20, 8, 8, 29)]

In [121]:
type(updated)
 

list

In [131]:
w=updated[0].date()


In [132]:
w

datetime.date(2013, 8, 20)

In [130]:
print(w)

<built-in method date of datetime.datetime object at 0x0000008501FD4418>


In [20]:


def mains(url):
    import pandas as pd
    check = [url_having_ip(url),url_length(url),url_short(url),having_at_symbol(url),doubleSlash(url),prefix_suffix(url),sub_domain(url),SSLfinal_State(url),domain_registration(url),https_token(url),request_url(url),url_of_anchor(url),Links_in_tags(url),sfh(url),email_submit(url),abnormal_url(url)]
    check1=pd.DataFrame(check)
    return check1
url="http://peegh1o.com/click?h=Ax722bagzrnQpadmvTyqRyZiGybhzQ0bWVanTmqex6bs3PXVsKZUeOi8j_Uh0mfP2QVZTrpC26VLQPK_YkAMMQhj7SjSqlxPvOZU9IYSJvVlzg5TFV1cmo_pukCcnrCBpMOmPKA69zAdDVCpH_RBeEaVeH7vPuIacQ6fgdeMh8b8MT16yGGR8Zan7VQSgPpBkf0AInnqTItbNDksmwoy6CPTQdv7PJSOWKI7-cz6-NZiLWoVXhi8yCtXTuNQKXPmswmgQ_NvRDR9HgT8tj2co-k42Vcw_wFFYgapaRnD1e-K-aOh5pl_Uyh74Vq812rDe7AbVmYdOJDHAItgUbccQzWphFZ-uwxUB2l5YLrIBlKc2v6quo9b84e984Nl8RkAqzSnEULiRw2mLVtVhbKJVZ4a-uuQmOQu40fJCfdho8FmgHEoQ5uKziPDf3voqzMd9FkOLhUy1mmOkbk0VlExRykkS8dFIHCuvSJ-wF2fwGaQhlGX2xo5Qc7USQnlRdMYxxoQxjS5gjd9sKjdyuJaN1shUbJ9nmU5Xq4QoOXzYY_UVPeUSiXvjMQmh3ZtSB-GpNY8rogrVHkvukGghjEhsiPpOeP9oJxkdn2D8rbXnt4nBDdQf4W_N58JS1DgMP1JvlsYeIr9auUsFpnOfpFKPhgNJGyGAdzvjNsMI-RDRRc92y-wrN9gvG_a7veKq-wepXZm3pU2_maMVwpzgINpzC-jhDJOvct6l0wLhQwMesLhJU2CC545Fcp2eEu8AVfvKRdaen58CwaMu42yAGKCpBsf8evPUfijBVGhaWZJhnZR8O_-jdSq_BfXQnnk4TMdEjzqAIhvpUIXOhtTS-9NcLqjcI0oNX44zGuHr8GXHxijnbSFLFo80FQBA3k3yoZZsuQ9DbXZ2pySmnkEgTXbWf4CxI9qZyzj2HyOWvXIRlcSjFc_npKoZQZ-KeXnwGBdsP8psa5b0fN-IPEDVUtF8VJbK87RMw9_aXS3chu_Dpc4RQJnTGSXxfMDd80b1HnqAO1WmrmQEQwlnI96siAlFG1WftNWJMzDE14BJwQgYHvfYUVSlijt1wf7r-6aMQdLYR0WtSjh_Hc3jNxdv7TCCvd87Cq3vZentvCwvc3LWjPCFhU9SMRkGPdAFr1Nonfm0mgjko11xYrxoPUVP-XB1KkVVnnB5ielsVN4kWniUIyOQBr_UirFAlLUF5Dabc8EsuTp1l9w6SC9tSf1rIZ53kPbBctSTbzaOJ3N2TPPukCV5Lf7Bt5kuVvMmAd-Jb8lApPiltD7fictRv3urR77L34b9nlh-xyZ3Bcr-KwjRyv34crZ8BbUeKNdbrbYEf3czulYq2yNMYugfskTMQ86E9iUCNirCfoWX4gcwcwd0tmUL2_-i6SpLB5_n0FkvnFpCc7pVJtOWu9bEqtXt9AHEPGXSYltnAqTnhbQWeumPTHsVIwPtMg-6U-0svsA4Ijv4Q-6TCcXvoEQwj0u5qX9TKCB2dr5uh-VgfDEYDYnoCDNRUUW2l_BkknrpWNwd5cFJzKuWv6T8nAKRGX89FVby14cEQel4Lz92WhbsxWXemA-jWpD9x7QkQXFdvEpcXagvCf2l7sk3w9PYLRVjNMQTf09J2IccepxZ7W1uX1B9PD7txI6eLFxoqRCtjKs-ZtI8YDlsKoblTmDQwaOSyhIoPrs4bZONgWNsz2IH_XgVbAzcYq7Lk0nLFMppAvDPUoeG4qce2UgQN9ZXyusAcgLUByy6lHWSi-DW__KCB0NAlFTQrAkRHvVIAunZdho9aqJ7vZJGbDTwXNzbRKj1TBh9FYUE7ClAX1HbsxHy9VZ6XqJMOhEOBmghGuyyJ77Pk2243AqFLE88YiEhH48GqLMmrPx0S5-i-ZSTm01-6jf9Q3FaDoLYkpTlba9HlPICnjyo7Ayz19HXHLuSbBQelJ5PD71LT8z7CcXa4sZ0wbfC5FciQE8ImBaMJGqmQPA28F-cG5d7vu9CRZFCtE5iV7WPUZHbv9hUNJG7T65GZNgLXo&subid=g-89168154-ebc7841502fa4ab59ef4529aa6508fc8-&data_test=2019041407_t&data_fb=no&data_rtt=1736&data_proto=http%3A&data_ic=false&fo=1&data_ss=728x1366&fo=1&t=&rt=11093&data_sid=2a4790e37c86271bc0bdbcd12c3ff70f&data_evt=focus&hr=1"
q=mains(url)
print(q)
type(q)


      0
0   1.0
1  -1.0
2   1.0
3   1.0
4   1.0
5   1.0
6   1.0
7  -1.0
8  -1.0
9  -1.0
10  1.0
11  1.0
12 -1.0
13  NaN
14  NaN
15  1.0


pandas.core.frame.DataFrame

In [35]:
import numpy
if numpy.isnan(q):
    q[13]=0
    q[14]=0
   

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [43]:
https_token(url)

1

In [39]:
mains(url)

webscraping.com


In [19]:
open("nanu","w+") 

<_io.TextIOWrapper name='nanu' mode='w+' encoding='cp1252'>

'C:\\Users\\vinu\\prjt'

In [31]:
url="http://webscraping.com/"

In [7]:
a=email_submit("http://tinyurl.com/ll59xhm")
a

In [8]:
a

'webscraping.com'

["Let's", 'Encrypt', 'Authority', 'X3']

"Let's"

In [44]:
startingDate

'Apr 24 23:52:04 2019 GMT'

c


In [5]:
a=redirect("https://drive.google.com/drive/my-drive")
a

0

In [17]:
total

1

''

0.0

1

[]

0